Loading dependencies, env variable and preparing flatten function

In [ ]:

from llama_index import GPTSimpleVectorIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext, Document
import os
from langchain.chat_models import ChatOpenAI
import logging
from llama_index.langchain_helpers.text_splitter import TokenTextSplitter
from llama_index import SimpleDirectoryReader, Document


def flatten(lst):
    """
    Flatten a list that contains other lists
    """
    flat_list = []
    for item in lst:
        if isinstance(item, list):
            flat_list.extend(flatten(item))
        else:
            flat_list.append(item)
    return flat_list


os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

Setting up text splitter and loading directories using readers.

In [ ]:
text_splitter = TokenTextSplitter(
    separator=" ", chunk_size=1024, chunk_overlap=200)

storytellerCommon = SimpleDirectoryReader(
    '/home/lukasz/workspace/storyteller/packages/storyteller/src/common', recursive=True, exclude=["**/*.spec.ts"]).load_data()
storytellerSrc = SimpleDirectoryReader(
    '/home/lukasz/workspace/storyteller/packages/storyteller/src/container', recursive=True, exclude=["**/*.spec.ts"]).load_data()
axiosStorytellerPlugin = SimpleDirectoryReader(
    '/home/lukasz/workspace/storyteller/packages/storyteller/src/plugins/axios', recursive=True, exclude=["**/*.spec.ts"]).load_data()
storytellerStorytellerPlugin = SimpleDirectoryReader(
    '/home/lukasz/workspace/storyteller/packages/storyteller/src/plugins/storyteller', recursive=True, exclude=["**/*.spec.ts"]).load_data()
typeormStorytellerPlugin = SimpleDirectoryReader(
    '/home/lukasz/workspace/storyteller/packages/storyteller/src/plugins/typeorm', recursive=True, exclude=["**/*.spec.ts"]).load_data()
dynamodbStorytellerPlugin = SimpleDirectoryReader(
    '/home/lukasz/workspace/storyteller/packages/storyteller/src/plugins/dynamodb', recursive=True, exclude=["**/*.spec.ts"]).load_data()
expressStorytellerPlugin = SimpleDirectoryReader(
    '/home/lukasz/workspace/storyteller/packages/storyteller/src/plugins/express', recursive=True, exclude=["**/*.spec.ts"]).load_data()
testsFramework = SimpleDirectoryReader(
    '/home/lukasz/workspace/storyteller/repositories/examples/5.1-real-life-scenario/tests/framework', recursive=True).load_data()
tests = SimpleDirectoryReader(
    '/home/lukasz/workspace/storyteller/repositories/examples/5.1-real-life-scenario/tests/user-needs-to-know', recursive=True).load_data()

Storing all documents in single index

In [ ]:
llm_predictor = LLMPredictor(llm=ChatOpenAI(
    temperature=0, model_name="gpt-4-32k"))
service_context = ServiceContext.from_defaults(
    llm_predictor=llm_predictor, chunk_size_limit=4096)
index = GPTSimpleVectorIndex.from_documents([], None, service_context)
documents = flatten(
    [axiosStorytellerPlugin,
     storytellerStorytellerPlugin,
     typeormStorytellerPlugin,
     dynamodbStorytellerPlugin,
     storytellerCommon,
     storytellerSrc,
     testsFramework, tests,
     expressStorytellerPlugin, tests])
for document in documents:
    logging.info(document.doc_id)
    index.insert(document)

Storing data

In [ ]:
index.save_to_disk('index.json')